In [ ]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import numpy as np
from scipy.constants import golden
import ipywidgets as widgets
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
import pytz
import os

In [ ]:
today = datetime.datetime.today()
# Select your local timezone to get the right date when running the notebook
localtimezone = pytz.timezone("US/Pacific")
today = today.astimezone(localtimezone)
print(f"{today.year}-{today.month:02d}-{today.day:02d}")

In [ ]:
filename = f"https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-{today.year}-{today.month:02d}-{today.day:02d}.xlsx"

In [ ]:
df = pd.read_excel(filename)

In [ ]:
df.head()

In [ ]:
def logistic_curve(x, a, b, c, d):
    """
    Logistic function with parameters a, b, c, d
    a is the curve's maximum value (top asymptote)
    b is the curve's minimum value (bottom asymptote)
    c is the logistic growth rate or steepness of the curve
    d is the x value of the sigmoid's midpoint
    """
    return ((a-b) / (1 + np.exp(-c * (x - d)))) + b

In [ ]:
def logarithmic_curve_fit(selectedcountry, save_figure=False):
    # Define image parameters
    image_width=8
    
    # initialize variables
    passed_inflection_point = False
    markersize = 10
    
    # Create figure with the right dimensions
    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(image_width*golden, image_width))
    
    # Filter the dataframe for the selected country
    # To avoid asymmetries, get only rows of the selected country with at least one case
    # Sort the dataframe by date
    countrydf = df[(df['cases'] >= 1) & (df['geoId']==selectedcountry)].sort_values(by='dateRep')
    # Extract the new cases in a variable, and make it an array for some calculations
    confirmed = countrydf['cases']
    confirmed_daily = confirmed.values
    # Get the name and population of the selected country
    population = df[df['geoId'] == selectedcountry]['popData2018'].values[-1]
    country_name = df[df['geoId'] == selectedcountry]['countriesAndTerritories'].values[0]

    # Do the calculations only if there are more than 10 confirmed cases for the selected country
    if len(confirmed) > 10:
        # Create the x values for the fit function: number of days since the first infection
        # The y-values are the cumulative number of cases: confirmed.cumsum()
        x = np.arange(0, len(confirmed))
        # Set the initial parameters for the fit function
        # a = current maximum value
        # b = 1
        # c = 0.1
        # d = half the number of days with new infections for the selected country
        p0 = [confirmed.values.sum(), 1, 0.1, len(confirmed_daily)//2]
        try:
            # actual fit to the logistic function
            logistic_params, covariance = curve_fit(logistic_curve, x, confirmed.cumsum().values, p0=p0)
            a, b, c, d = logistic_params
        except Exception as e1:
            print(e1)
            return 0
        
        # Detect if the inflection point has been reached
        if d > 0 and d < len(confirmed):
            passed_inflection_point = True
        else:
            days_to_inflection = int(d)-len(confirmed)+1
            plural = f"{'s' if days_to_inflection > 1 else ''}"
            
        # plot the actual cases reported in the database as blue dots
        axs.plot(np.arange(0, len(confirmed)), confirmed.cumsum().values, ".", markersize=8, label="Reported cases")
        
        # Calculate how good is the fit
        confirmed_pred = logistic_curve(np.arange(0, len(confirmed)), *logistic_params)
        r2 = r2_score(confirmed.cumsum().values, confirmed_pred)
        
        # Plot R2 rendering LaTeX on the plot
        axs.text(0.2, 0.3, f"$R^2$={r2:.4f}", transform=axs.transAxes)
    
        # calculate when the max number of cases increases so little new cases are not increasing
        # this provides a slightly more accurate value than simply doubling the days of the inflection point
        # uncomment the following lines if you want to see the difference
        """
        days_pred = np.arange(len(confirmed)-(len(confirmed)%15), 360, 15)
        ypred = logistic_curve(days_pred, *logistic_params)
        
        diff_ypred = [ypred[i + 1] - ypred[i] for i in range(len(ypred)-1)]
        days_diff = days_pred[np.argmax(diff_ypred < np.max(diff_ypred)*0.000001)] - len(confirmed)
        
        x = np.arange(0, days_pred[np.argmax(diff_ypred < np.max(diff_ypred)*0.000001)])
        y = logistic_curve(x, *logistic_params)
        """
        # Calculate the whole curve using the double of days than the inflection point
        # Comment the next 3 lines if you are testing the calculation above
        x = np.arange(0, np.ceil(d*2))
        y = logistic_curve(x, *logistic_params)
        y_inflection_point = logistic_curve(int(d), *logistic_params)
        days_diff = int(d) - len(confirmed)
        days_to_asymptote = int(d*2) - len(confirmed)

        # plot a vertical black dotted line indicating the end of the actual data
        axs.vlines(len(confirmed_daily), 0, max(max(y), confirmed_daily.sum()), color='k', ls=':', label="Today")#, transform=axs.transAxes)
        
        # generate the inflection point label
        inflection_point_label = f"Inflection point{' in ' if not passed_inflection_point else ' '}{abs(days_diff+1 )} days{' ago' if passed_inflection_point else ''}"
        # plot a vertical red dotted line indicating the location of the inflection point as a reference
        axs.vlines(int(d), 0.0, max(max(y), confirmed_daily.sum()), color='r', ls=':', label=inflection_point_label)#, transform=axs.transAxes)
        
        # annotate the inflection point as a reference
        axs.text(int(d)+1, y_inflection_point, f"{int(d)} days")
        # plot the fitted curve in orange
        axs.plot(x, y, color='#fc5e03', label="Fitted logistic function")

        # plot the predicted maximum number of cases and the number of days until then
        text_x = len(confirmed)+2
        text_y = 0.05*max(max(y), confirmed_daily.sum())
        axs.text(text_x, text_y, f"Day {len(confirmed)}") 
        if confirmed.cumsum().values[-1] < y[-1]:
            text_x = 0.7
            text_y = 0.01
            axs.text(text_x, text_y, f"Cases in {abs(days_to_asymptote)} days from today: {int(y[-1]):,}", transform=axs.transAxes)

        axs.fill_between(x[len(confirmed):], confirmed.cumsum().values[-1], y[len(confirmed):],
                         facecolor="none", hatch="/", edgecolor=f"{'b' if passed_inflection_point else 'r'}", linewidth=0.0,
                         label=f"Predicted increase")

    
    axs.set_xlabel('Days since first infection')
    axs.legend(loc='upper left')
    axs.set_title('US COVID-19 cases')
    
    if save_figure:
        filename = f"{selectedcountry}_logistic_fit_{today.year}-{today.month:02d}-{today.day:02d}.png"
        destination_folder = os.path.expanduser(r"~\Documents")
        if not os.path.isdir(destination_folder):
            try:
                os.mkdir(destination_folder)
            except Exception as e:
                print(e)
        plt.savefig(os.path.join(destination_folder, filename), dpi=300, bbox_inches='tight')
        print(f"file saved as: {os.path.join(destination_folder, filename)}")


In [ ]:
countries = df.loc[df[df['cases'] >= 1].index,['countriesAndTerritories', 'geoId', 'cases']].groupby(['countriesAndTerritories', 'geoId'], as_index=False)['cases'].sum().values

countries = countries[countries[:,2].argsort()[::-1]]

countries = [(f"{a} ({c})", b) for a, b, c in countries]

%matplotlib inline
style = {'description_width': 'initial'}
output = widgets.interact(logarithmic_curve_fit,
                          selectedcountry=widgets.Dropdown(options=countries,
                                                           value=countries[0][1],
                                                           description='Country (total):',
                                                           style=style),
                          save_figure=widgets.Checkbox(False, disabled=False))

# Plot sample logistic function

In [ ]:
a = 10
b = 0
c = 1
d = 0
x = np.arange(-10, 10, 0.1)
y = logistic_curve(x, a, b, c, d)

In [ ]:
fig = plt.figure(figsize=(15, 10))
plt.plot(d, logistic_curve(d, a, b, c, d), 'or')
plt.vlines(d, 0, 10, 'k', ':')
plt.hlines(logistic_curve(d, a, b, c, d), x[0], x[-1], 'k', ':')
# s = r"$f(x) = \frac{a}{1+e^(-c (x-d))}$"
# s = s+"\na=10, b=0, c=1, d=0"
# plt.text(-10, 8, s, fontdict=dict(fontsize='large'))
plt.annotate('inflection point', xy=(d, logistic_curve(d, a, b, c, d)), xytext=(4, 6),
             arrowprops=dict(arrowstyle='-|>',facecolor='black'))
plt.plot(x, y)
# plt.savefig(os.path.join(os.path.expanduser('~\Documents'), 'sigmoid.png'), bbox_inches='tight', dpi=300)